### Notebook to Research diabetes onset 


#### Matched Controls Diabetes Onset Curve Construction Details

   1. <div style="background:Aliceblue"> <strong>Get all the canary patients who have their CC_DIABETES flag equals 0
    on their registration date.</strong>
   </div>
   
       - Pick a matching algorithm
       
   2. <div style="background:Aliceblue"><strong> Create the macthed population</strong> </div>
   
       - For each canary patient let their registration month be $i$
       - Find N(for example 50)  control patients to match this canary patient from the pmpm 
       patients on month i who have CC_DIABETES = 0 on month i. For example a matching specification could require the N patients should have the same age 
       and then be the nearest neighbors according to other features(log costs, cc codes etc).
       - Set the index date for these pmpm controls patients to be month $i$.
       - This forms a matched set of roughly 50 times the number of canary patients and each control has an index date.
   
   3. <div style="background:Aliceblue"><strong> Compute Onset of Diabetes for month $i$<strong></div> 
   
       - Get all the matched controls with index date prior to month $i$.
       - For Month i we define the total number of Diabetes Onsets(DO) as the number of subjects from <strong> 2 </strong> that had
        the flag $CC_{DIABETES} = 1$ at Month $(i)$ and $CC_{DIABETES} = 0$ at Month $(i-1)$ (Note these patients must be in pmpm on both months).
       - S is the total number of matched controls included in the pmpm both in month $i$ and month $i-1$
       - For Month i we define the diabetes onset rate(DOR) as: $DOR =  1000DO/S$

    <div style="background:Lightskyblue"> <strong>Suggestion:</strong> Remove from  $S$ the subjects that  have the flag $CC_{DIABETES} = 1$ at month $i-1$
    </div>

   

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython

In [2]:
%reload_ext Cython

In [3]:
import sys
sys.path.append('..')
import warnings
def warn(message = None, category=None, stacklevel=1, source=None):
    pass

warnings.warn = warn

In [16]:
path = str(Path.home()) + '/rpi-canary-eval/temp'
git_path = str(Path.home()) + '/rpi-canary-eval'
sys.path.append(git_path + '/modules')

In [43]:
import scipy.stats as sts
import pandas as pd
import numpy as np
import time
import datetime as dt
from pathlib import Path
import time
#custom
import data_process
from data_functions import make_date_to_MYR
from diabetes_onset import( process_canary,
get_tc, in_canary, diabet_onset, not_first_month_diabetes, 
                           match_data_per_month, 
                           filter_first_month_diabetes,
                           matched_population, 
                           propensity_func,
                           nearest_neighbor_func, 
                           random_matching_func, agglomerative )
from path import path_dict, col_seq_cc, col_seq_can, col_seq_cc_all, col_seq_can_all
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(seed = 0)

### Load Pmpm, Canary data

In [6]:
%%time
canary_p = '../../../data/CANARY'
pmpm_p = '../../../temp/CC_PMPM.p'
preg_p = '../../../data/pregnancies'

preg = pq.read_table(preg_p).to_pandas()
canary = pq.read_table(canary_p).to_pandas()
pmpm = pd.read_pickle(pmpm_p)

#drop pmpm duplicates
pmpm.drop_duplicates(subset = ['PERS_ID', 'MYR'], inplace = True)
canary.drop_duplicates(subset = ['PERS_ID'], inplace = True)

CPU times: user 7.03 s, sys: 8.69 s, total: 15.7 s
Wall time: 36.9 s


In [7]:
pmpm.MEDICAID_LOB.unique()

array(['Medicare', 'Commercial', 'Other', 'ASO', 'Medicaid-HARP',
       'Medicaid-TANF', 'Medicaid-SSI', 'Medicaid-CHP', 'Medicaid-Other'],
      dtype=object)

### Include Line of Bussiness Feature 1 Not medicaid 0 medicaid

In [8]:
medicaid_mask = pmpm.MEDICAID_LOB.str.contains(r'.*Medicaid-.*')
pmpm['LOB'] = 1
pmpm.loc[medicaid_mask, 'LOB'] = 0

In [20]:
pmpm['MEDICAID_LOB'] = pmpm['LOB']
pmpm = pmpm.drop(['LOB'], axis=1)

In [19]:
pmpm.ALLOW_AMT = pmpm.ALLOW_AMT.apply(data_process.log_)
pmpm.ALLOW_IP = pmpm.ALLOW_IP.apply(data_process.log_)
pmpm.ALLOW_ER = pmpm.ALLOW_ER.apply(data_process.log_)

In [9]:
preg = preg.rename(mapper = {'person_id':'PERS_ID'}, axis = 1)
preg = pd.melt(preg, id_vars = ['PERS_ID'], value_vars=['stay_start_date', 'stay_end_date'], value_name='MYR')
preg.MYR = preg.MYR.apply(lambda x: x.year*100+x.month)
preg.drop(labels = 'variable', axis = 1, inplace = True)
preg['PREGNANCY'] = 1

In [10]:
%%time
pmpm = pmpm.merge(preg, how = 'left', on = ['PERS_ID', 'MYR'])
pmpm.PREGNANCY = pmpm.PREGNANCY.fillna(value = 0)
pmpm.drop_duplicates(subset = ['PERS_ID', 'MYR'], inplace = True)

CPU times: user 13 s, sys: 8 s, total: 21 s
Wall time: 21 s


In [10]:
# from utils import ip_er2

In [11]:
%%time
pmpm = pmpm.sort_values(by = ['PERS_ID', 'MYR']).reset_index(drop = True)
pmpm['IP_BIN'] = ((pmpm.ALLOW_IP>0) & (pmpm.PREGNANCY == 0)).astype(int)
pmpm['ER_BIN'] = ((pmpm.ALLOW_ER>0) & (pmpm.PREGNANCY == 0)).astype(int)
pmpm['ACUTE'] =  ((pmpm.IP_BIN>0) | (pmpm.ER_BIN > 0 )).astype(int)

CPU times: user 7.58 s, sys: 2.97 s, total: 10.6 s
Wall time: 10.6 s


In [12]:
%%time
pmpm[['IP_C', 'ER_C', 'ACUTE_C']] = pmpm.groupby('PERS_ID', sort = False)[['IP_BIN', 'ER_BIN', 'ACUTE']].cumsum().reset_index(drop = True)
pmpm[['IP_BIN6', 'ER_BIN6', 'ACUTE6']] = pmpm.groupby('PERS_ID', sort = False)[['IP_C', 'ER_C', 'ACUTE_C']].diff(6).fillna(pmpm[['IP_C', 'ER_C','ACUTE_C']]).reset_index(drop = True)
pmpm[['IP_BIN2', 'ER_BIN2','ACUTE2']] = pmpm.groupby('PERS_ID', sort = False)[['IP_C', 'ER_C','ACUTE_C']].diff(2).fillna(pmpm[['IP_C', 'ER_C','ACUTE_C']]).reset_index(drop = True)

CPU times: user 3min 58s, sys: 10.3 s, total: 4min 8s
Wall time: 4min 3s


#undersample
test = test.sample(random_state = 0, frac = 0.01).reset_index(drop = True)
test = test.sort_values(by = ['PERS_ID', 'MYR']).reset_index(drop = True)

%%time

test[['IP_C', 'ER_C']] = test.groupby('PERS_ID', sort = False)[['IP_BIN', 'ER_BIN']].cumsum().reset_index(drop = True)
test[['IP_BIN2', 'ER_BIN2']] = test.groupby('PERS_ID', sort = False)[['IP_C', 'ER_C']].diff(6).fillna(test[['IP_C', 'ER_C']]).reset_index(drop = True)

In [21]:
pmpm.head()

PERS_ID     MYR  MEDICAID_LOB  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  \
0      -3  201908             1              119        0.0       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
1      -3  201909             1              119        0.0       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
2      -3  201910             1              119        0.0       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
3      -3  201911             1              119  

### Process Canary data 
1. Drop register dates beyond pmpm max date
2. Transform register date to MYR
3. Keep only ids and MYR

### Get Controls and Treated
> ***a)Controls are all the patients that did not enrolled in canary program at any point  
b)Treated are the patients that enrolled at some point in the Canary Plan***

In [22]:
%%time
canary2 = process_canary(canary, pmpm.MYR.max())
treated, controls = get_tc(pmpm.copy(), canary2)

CPU times: user 8.66 s, sys: 4.14 s, total: 12.8 s
Wall time: 12.8 s


### filter by principal reason

In [23]:
controls.PERS_ID.unique().shape, treated.PERS_ID.unique().shape

Keep Canary Patients from their date of registration and onwards who did not have **CC_DIABETES** flag on, on their date of registration.

In [24]:
%%time
treated2 = in_canary(treated.copy(), canary2)
#treated2 = filter_first_month_diabetes(treated2.copy())
treated2.drop(columns = ['IN_CANARY'], inplace = True)

CPU times: user 5.12 s, sys: 99 ms, total: 5.22 s
Wall time: 5.22 s


In [31]:
treated2.PERS_ID.unique().shape

In [65]:
#cols = list(col for col in treated2.columns if col not in ['PERS_ID', 'MYR'])
#columns_to_match = cols
columns_to_match = ['AGE_AT_MIDMONTH', 'ALLOW_AMT', 'MBR_GNDR','CC_TOBACCO', 'CC_PRESSURE', 'CC_OBESITY', 'MEDICAID_LOB',
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID', 'CC_DX_CNT', 'IP_BIN2', 'ER_BIN2', 'ACUTE', 'ACUTE2', 'ACUTE6', 'IP_BIN6', 'ER_BIN6']

In [38]:
onset = "CC_DIABETES"
cols_to_keep = ["PERS_ID", "MYR", "AGE_AT_MIDMONTH", "MBR_GNDR", onset]

In [33]:
import tensorflow as tf
from tensorflow.keras import models, layers
import warnings
warnings.filterwarnings('ignore')

model = models.load_model(path+"/TF_Latent_Model_m6_Canary_stacked", compile=False)

In [34]:
latent_cols = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16']

In [39]:
%%time
#transform data of sick and healthy treated, and controls
tr2 = data_process.transform_features_with_NN(model, treated2[col_seq_can_all].copy(), cols_to_keep, latent_dim=16)
cntrls = data_process.transform_features_with_NN(model, controls[col_seq_can_all].copy(), cols_to_keep, latent_dim=16)

CPU times: user 1min 40s, sys: 25.6 s, total: 2min 6s
Wall time: 1min 12s


In [40]:
tr2.head()

PERS_ID     MYR  AGE_AT_MIDMONTH  MBR_GNDR  CC_DIABETES        x1        x2        x3        x4        x5        x6        x7        x8        x9       x10       x11       x12       x13       x14      x15       x16
0  2602482  201908               81         0          0.0 -0.057048 -0.058682 -0.056749  0.051557  0.051478 -0.060372 -0.079283  0.066987 -0.048584 -0.027853 -0.031246 -0.034941  0.019237 -0.039246 -0.02541  0.077789
1  2602482  201909               81         0          0.0 -0.057048 -0.058682 -0.056749  0.051557  0.051478 -0.060372 -0.079283  0.066987 -0.048584 -0.027853 -0.031246 -0.034941  0.019237 -0.039246 -0.02541  0.077789
2  2602482  201910               81         0          0.0 -0.057048 -0.058682 -0.056749  0.051557  0.051478 -0.060372 -0.079283  0.066987 -0.048584 -0.027853 -0.031246 -0.034941  0.019237 -0.039246 -0.02541  0.077789
3  2602482  201911               82         0          0.0 -0.057048 -0.058682 -0.056749  0.051557  0.051478 -0.060372 -0.079283  0.066987 -0.048584 -0.027853 -0.031246 -0.034941  0.019237 -0.039246 -0.02541  0.077789
4  2602482  201912               82         0          0.0 -0.057048 -0.058682 -0.056749  0.051557  0.051478 -0.060372 -0.079283  0.066987 -0.048584 -0.027853 -0.031246 -0.034941  0.019237 -0.039246 -0.02541  0.077789

In [44]:
%%time

#columns_to_match = columns_to_match + ['IP_BIN2', 'ER_BIN2']
#hier_cols = ['IP_BIN2', 'ER_BIN2', 'ACUTE2', 'ACUTE6', 'IP_BIN6', 'ER_BIN6']
# hier_cols = ['CC_DX_CNT']
# hier_range = [1]
# exact_match_dict = {'ACUTE':None, 'ACUTE6':None, 'ACUTE2':None, 'LOB':None}
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None}
matched0, matched_data0 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)

"""
#columns_to_match = columns_to_match + ['IP_BIN2', 'ER_BIN2']
propensity_func_args = {'match_on':columns_to_match, 'neighbors_per_treated': 5, 'onset': "CC_DIABETES", "onset_val" = 1}
matched0, matched_data0 = matched_population(treated2.copy(), 
                                           controls.copy(),
                                           propensity_func, 
                                           propensity_func_args)
"""


Mathcing Function: nearest_neighbor_func
hERE
After filtering control ids are:328522 before they were:354792


1604it [29:19,  1.10s/it]

CPU times: user 44min 43s, sys: 27min 10s, total: 1h 11min 54s
Wall time: 29min 23s


'\n#columns_to_match = columns_to_match + [\'IP_BIN2\', \'ER_BIN2\']\npropensity_func_args = {\'match_on\':columns_to_match, \'neighbors_per_treated\': 5, \'onset\': "CC_DIABETES", "onset_val" = 1}\nmatched0, matched_data0 = matched_population(treated2.copy(), \n                                           controls.copy(),\n                                           propensity_func, \n                                           propensity_func_args)\n'

####  In addition to the previously created matched groups create a random group

In [50]:
Path('./tempHybridSelectionMoreData').mkdir(exist_ok = True)
p = './tempHybridSelectionMoreData/'

In [51]:
random_matching_func_args = {'neighbors_per_treated': 5}
random_controls = matched_population(tr2.copy(), 
                                    cntrls.copy(),
                                    random_matching_func, 
                                    random_matching_func_args)

Mathcing Function: random_matching_func


In [52]:
matched_data0.to_pickle(p+'match0_l.p')
random_controls.to_pickle(p+'random_l.p')
tr2.to_pickle(p+'treated0_l.p')

In [53]:
matched_data0_o = controls[controls.set_index(["PERS_ID", "MYR"]).index.isin(matched_data0.set_index(["PERS_ID", "MYR"]).index)].sort_values(by=["PERS_ID", "MYR"])
random_controls_o = controls[controls.set_index(["PERS_ID", "MYR"]).index.isin(random_controls.set_index(["PERS_ID", "MYR"]).index)].sort_values(by=["PERS_ID", "MYR"])

In [54]:
matched_data0_o.to_pickle(p+'match0_o.p')
random_controls_o.to_pickle(p+'random_o.p')
treated2.to_pickle(p+'treated0_o.p')

In [57]:
import os
files_list = os.listdir(p)

#save in S3
import boto3
s3 = boto3.client('s3')
bucket = 'cdphp-s3-us-e-p-pond'
for file in files_list:
    s3.upload_file( p +file,
                    bucket, 
                    'rpi/nneehal/DATA/m6_CAN_Stacked_202108/'+file,
                    ExtraArgs = {'ACL':"bucket-owner-full-control", 'ServerSideEncryption':'AES256'})

In [56]:
files_list

['match0_l.p',
 'random_l.p',
 'treated0_l.p',
 'match0_o.p',
 'random_o.p',
 'treated0_o.p']

#### Calculate the Diabetes onset for each of the created groups
1. Group 0: Group of treated patients cluster 0
2. Group 1: Group of treated patients Cluster 1
3. Group 2: Group of matched control patients to Group 0
4. Group 3: Group of matched control patients to Group 1
5. Group 4: Random Group

In [58]:
from utils import statistics_first_month

In [59]:
pmpm2 = pmpm
try:
    pmpm2.MBR_GNDR.replace(to_replace = {'F':0, 'M':1}, inplace = True)
except:
    pass

In [66]:
%%time
tr_desc0, Ntr, treated_first_month = statistics_first_month(treated2, columns_to_match, return_array = True)
tr_desc0 = tr_desc0.T



con_desc0, Ncon, controls_first_month = statistics_first_month(matched_data0_o, columns_to_match, return_array = True)
#con_desc.index = pd.MultiIndex.from_product([['Matched'], con_desc.index])
con_desc0 = con_desc0.T

pmpm_desc, Npmpm = statistics_first_month(pmpm2, columns_to_match)
#pmpm_desc.index = pd.MultiIndex.from_product([['PMPM'], pmpm_desc.index])
pmpm_desc = pmpm_desc.T

random_desc, Nra = statistics_first_month(random_controls_o, columns_to_match)
#pmpm_desc.index = pd.MultiIndex.from_product([['PMPM'], pmpm_desc.index])
random_desc = random_desc.T

CPU times: user 2min 29s, sys: 4.69 s, total: 2min 34s
Wall time: 2min 34s


In [67]:
import scipy.stats as sts

In [68]:
args = {'mean1':tr_desc0['mean'], 'std1':tr_desc0['std'], 'mean2':con_desc0['mean'], 'std2':con_desc0['std'], 'nobs1': np.full(len(tr_desc0), Ntr) , 'nobs2': np.full(len(tr_desc0), Ncon)}
stats, pval = sts.ttest_ind_from_stats(**args)
stats = pd.DataFrame(stats, columns = ['t-stat'])
stats['pval'] = pval

In [69]:
alli = pd.concat((tr_desc0['mean'], con_desc0['mean'], stats, pmpm_desc['mean'], random_desc['mean']), axis = 1, keys = ['Treated', 'Matched Controls', 'T-Test', 'Genaral Population', 'Random'] )

In [70]:
alli

Treated Matched Controls     T-Test               Genaral Population     Random
                      mean             mean     t-stat          pval               mean       mean
AGE_AT_MIDMONTH  50.205736        50.333196  -0.356769  7.212732e-01          44.310976  47.947482
ALLOW_AMT         4.273698         4.085187   2.430449  1.509981e-02           4.687940   3.716769
MBR_GNDR          0.205736         0.217780  -1.061705  2.883985e-01           0.442749   0.420270
CC_TOBACCO        0.061097         0.122149  -7.061446  1.771716e-12           0.074202   0.097533
CC_PRESSURE       0.001247         0.003985  -1.683051  9.240036e-02           0.007390   0.006867
CC_OBESITY        0.456359         0.275900  14.326000  4.895785e-46           0.108841   0.229400
MEDICAID_LOB      0.951995         0.825227  12.923545  7.267407e-38           0.755775   0.791709
CC_HYPERTENSION   0.324813         0.238115   7.243280  4.744920e-13           0.207717   0.253815
CC_HYPOTHYROID    0.094763         0.076257   2.479254  1.318416e-02           0.047300   0.064344
CC_DX_CNT         2.706359         2.431987   4.421709  9.908830e-06           1.824419   2.417726
IP_BIN2           0.010599         0.012641  -0.632583  5.270225e-01           0.020587   0.015895
ER_BIN2           0.029925         0.028854   0.222487  8.239398e-01           0.047286   0.040565
ACUTE             0.015586         0.016625  -0.296327  7.669871e-01           0.063258   0.027085
ACUTE2            0.038653         0.039984  -0.233430  8.154328e-01           0.063258   0.053917
ACUTE6            0.111596         0.133691  -1.810618  7.023372e-02           0.063258   0.155264
IP_BIN6           0.028055         0.035999  -1.270494  2.039420e-01           0.020587   0.046414
ER_BIN6           0.088529         0.103325  -1.405381  1.599430e-01           0.047286   0.116862

## Statistics on matching date

#### Univariate Means of the Groups under Study of interesting features (Statistics for whole populations)
>The "Sick groups" have:  
a) Older age  
b) More Expenditures  
c) Smoke More  
d) Have More People with Hypertension


In [71]:
tr_desc0A = treated2[columns_to_match].describe().loc[['mean', 'std']]
#tr_desc.index = pd.MultiIndex.from_product([['Treated'], tr_desc.index])
tr_desc0A = tr_desc0A.T


con_desc0A = matched_data0_o[columns_to_match].describe().loc[['mean', 'std']]
#con_desc.index = pd.MultiIndex.from_product([['Matched'], con_desc.index])
con_desc0A = con_desc0A.T


pmpm_descA = pmpm2[columns_to_match].describe().loc[['mean','std']]
#pmpm_desc.index = pd.MultiIndex.from_product([['PMPM'], pmpm_desc.index])
pmpm_descA = pmpm_descA.T

random_descA = random_controls_o[columns_to_match].describe().loc[['mean','std']]
#pmpm_desc.index = pd.MultiIndex.from_product([['PMPM'], pmpm_desc.index])
random_descA = random_descA.T



In [72]:
args = {'mean1':tr_desc0A['mean'], 'std1':tr_desc0A['std'],
        'mean2':con_desc0A['mean'], 'std2':con_desc0A['std'],
        'nobs1': np.full(len(tr_desc0), len(treated2)) , 'nobs2': np.full(len(tr_desc0), len(matched_data0))}
stats, pval = sts.ttest_ind_from_stats(**args)
stats = pd.DataFrame(stats, columns = ['t-stat'])
stats['pval'] = pval

In [73]:
alli2 = pd.concat((tr_desc0A, con_desc0A, stats['pval'], pmpm_descA['mean'], random_descA['mean']), axis = 1, keys = ['Treated',
                                                                                 'Matched Controls','T-Test',
                                                                                'Genaral Population', 
                                                                                 'Random'] )
alli2

Treated            Matched Controls                    T-Test Genaral Population     Random
                      mean        std             mean        std           pval               mean       mean
AGE_AT_MIDMONTH  50.837442  12.502708        51.104446  12.692507   2.216635e-03          50.378855  50.334805
ALLOW_AMT         4.162768   2.793698         3.982334   2.865661   4.557813e-20           4.007592   3.800661
MBR_GNDR          0.189401   0.391834         0.200071   0.400055   1.030291e-04           0.433015   0.418230
CC_TOBACCO        0.063364   0.243621         0.123588   0.329112  2.761972e-169           0.101510   0.104053
CC_PRESSURE       0.002304   0.047947         0.003492   0.058989   2.525276e-03           0.012810   0.008006
CC_OBESITY        0.504570   0.499989         0.325688   0.468634   0.000000e+00           0.255020   0.283281
MEDICAID_LOB      0.953034   0.211571         0.823824   0.380972   0.000000e+00           0.802084   0.811097
CC_HYPERTENSION   0.325346   0.468513         0.266216   0.441981   7.588812e-82           0.328768   0.298122
CC_HYPOTHYROID    0.095084   0.293337         0.083422   0.276520   1.502734e-09           0.078325   0.074747
CC_DX_CNT         2.917857   2.277962         2.700227   2.359947   2.746554e-41           2.954872   2.766443
IP_BIN2           0.008257   0.091756         0.012679   0.119470   2.232499e-08           0.020239   0.014544
ER_BIN2           0.024923   0.168901         0.033136   0.189270   1.363483e-10           0.043882   0.035101
ACUTE             0.015515   0.123590         0.021418   0.144773   1.251360e-09           0.030728   0.023083
ACUTE2            0.031951   0.189130         0.043768   0.219786   1.258509e-15           0.060496   0.047162
ACUTE6            0.099539   0.402286         0.133049   0.443761   7.878799e-29           0.167558   0.142737
IP_BIN6           0.025000   0.168889         0.037341   0.236495   1.847474e-15           0.054964   0.042749
ER_BIN6           0.078648   0.367719         0.101868   0.375598   2.244951e-19           0.122475   0.107313

In [77]:
random_controls.head()

PERS_ID     MYR  AGE_AT_MIDMONTH  MBR_GNDR  CC_DIABETES        x1        x2        x3        x4        x5        x6        x7        x8        x9       x10       x11       x12       x13       x14      x15       x16
0  2828660  201710               56         0          0.0 -0.057048 -0.058682 -0.056749  0.051557  0.051478 -0.060372 -0.079283  0.066987 -0.048584 -0.027853 -0.031246 -0.034941  0.019237 -0.039246 -0.02541  0.077789
1  2828660  201711               56         0          0.0 -0.057048 -0.058682 -0.056749  0.051557  0.051478 -0.060372 -0.079283  0.066987 -0.048584 -0.027853 -0.031246 -0.034941  0.019237 -0.039246 -0.02541  0.077789
2  2828660  201712               56         0          0.0 -0.057048 -0.058682 -0.056749  0.051557  0.051478 -0.060372 -0.079283  0.066987 -0.048584 -0.027853 -0.031246 -0.034941  0.019237 -0.039246 -0.02541  0.077789
3  2828660  201801               56         0          0.0 -0.057048 -0.058682 -0.056749  0.051557  0.051478 -0.060372 -0.079283  0.066987 -0.048584 -0.027853 -0.031246 -0.034941  0.019237 -0.039246 -0.02541  0.077789
4  2828660  201802               56         0          0.0 -0.057048 -0.058682 -0.056749  0.051557  0.051478 -0.060372 -0.079283  0.066987 -0.048584 -0.027853 -0.031246 -0.034941  0.019237 -0.039246 -0.02541  0.077789

In [78]:
columns_to_match = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16']

tr_desc0 = tr2[columns_to_match].describe().loc[['mean','std']].T
con_desc0 = matched_data0[columns_to_match].describe().loc[['mean','std']].T
random_desc = random_controls[columns_to_match].describe().loc[['mean','std']].T

alli_latent = pd.concat((tr_desc0, con_desc0, random_desc), axis = 1, keys = ['Treated', 'Matched Controls', 'Random'] )

In [79]:
alli_latent

Treated               Matched Controls                  Random              
         mean           std             mean           std      mean           std
x1  -0.057048  1.463229e-08        -0.057048  1.552562e-08 -0.057048  1.858295e-08
x2  -0.058682  4.979188e-09        -0.058682  5.450406e-09 -0.058682  6.797000e-09
x3  -0.056749  1.189825e-08        -0.056749  1.260848e-08 -0.056749  1.473899e-08
x4   0.051557  3.277393e-09         0.051557  3.406905e-09  0.051557  3.862245e-09
x5   0.051478  2.836381e-09         0.051478  3.107252e-09  0.051478  3.164127e-09
x6  -0.060372  4.760468e-09        -0.060372  5.034142e-09 -0.060372  5.667104e-09
x7  -0.079283  1.977318e-08        -0.079283  2.092987e-08 -0.079283  2.500898e-08
x8   0.066987  8.139627e-09         0.066987  8.580747e-09  0.066987  9.673818e-09
x9  -0.048584  1.218917e-08        -0.048584  1.292479e-08 -0.048584  1.533829e-08
x10 -0.027853  4.966259e-09        -0.027853  5.286304e-09 -0.027853  6.578119e-09
x11 -0.031246  1.077953e-08        -0.031246  1.146131e-08 -0.031246  1.390998e-08
x12 -0.034941  4.103006e-09        -0.034941  4.455826e-09 -0.034941  5.499033e-09
x13  0.019237  4.390356e-09         0.019237  4.655122e-09  0.019237  5.603183e-09
x14 -0.039246  9.173468e-09        -0.039246  9.790524e-09 -0.039246  1.234710e-08
x15 -0.025410  7.222983e-09        -0.025410  7.677450e-09 -0.025410  8.958747e-09
x16  0.077789  3.951141e-09         0.077789  4.379281e-09  0.077789  5.608876e-09

#### Distribution Plots For Treated and Matched Controls
1)  Sick(S) Treated versus  Sick Control groups  
2) Healthy(H) Treated versus Healthy Control Groups

### Matrices and Figures for the paper

In [ ]:
#mapping = dict(zip(columns_to_match, ['Age', 'Total Cost', 'Gender', 'Tobacco use', 'Pressure', 'Obesity', 
                                     #'Hypertension', 'Hypothyroid', 'Disease Count', 'Inpatient Visits', 'Emergency Vistits', 'Line of Bussiness']))
mapping = dict(zip(columns_to_match, columns_to_match))
new_names = list(mapping.values())

In [ ]:
p = Path('./res/figs')
p.mkdir(parents = True, exist_ok = True)

In [ ]:
import matplotlib as mpl
sns.set_context(context = 'paper')
mpl.style.use('seaborn-paper')

1. treated_first_month
2. controls_first_month

In [ ]:
from diabetes_onset_plotter import plot_hist


In [ ]:
dist_plot_args = {'hist':True}
axes = plot_hist(treated_first_month.copy(), 
                 controls_first_month.copy(), mapping, ['ALLOW_AMT'] , 
                 data3 = None, 
                 save_path = 'res/figs/',
                dist_plot_args = dist_plot_args)

Save Three arrays
1. All Statistics
2. Only Means
3. Difference T-Statistic Pvalues

In [ ]:
from diabetes_onset_plotter import save_arrays
p2 = Path('./res/tabs')
p2.mkdir(parents = True, exist_ok = True)
paths = ['./res/tabs/all_stats.csv', './res/tabs/only_means.csv', './res/tabs/only_stats.csv']
(all_paper, only_means, only_stats) = save_arrays(alli, paths, mapping)

In [ ]:
all_paper


In [ ]:
only_means

In [ ]:
only_stats

In [ ]:
pmpm.MYR.max()